In [0]:
source_catalog = "adventureworks"
source_schema = "saleslt"      # confirmé
target_catalog = "ngow_lakehouse"
target_schema = "bronze"

# Tables à exclure
excluded = ["ErrorLog", "BuildVersion"]

# Récupération dynamique des tables du schema
tables_df = spark.sql(f"""
    SELECT table_name
    FROM {source_catalog}.information_schema.tables
    WHERE table_schema = '{source_schema}'
    AND table_name NOT IN ({','.join([f"'{t}'" for t in excluded])})
""")

tables = [row.table_name for row in tables_df.collect()]

print("📂 Tables détectées dans adventureworks.saleslt :", tables)

for table in tables:
    source_fullname = f"{source_catalog}.{source_schema}.{table}"
    target_fullname = f"{target_catalog}.{target_schema}.{table.lower()}"  # standardisé en minuscule
    
    print(f"➡️ Copie : {source_fullname} → {target_fullname}")
    
    df = spark.table(source_fullname)
    
    (df.write
       .format("delta")
       .mode("overwrite")     # Bronze = réécriture complète
       .saveAsTable(target_fullname)
    )

print("✔️ Toutes les tables ont été copiées dans la couche Bronze !")
